# 1 下载观测和导航数据

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


全局变量定义区，包含：

处理数据起止日期

测站名称

测站国家和站点信息

测站经纬高

信号类型

高度截至角

数据存储路径


从IGS官网下载完整测站信息list



In [1]:
import requests

# 目标URL
url = "https://network.igs.org/api/public/stations/?current=true&draw=1&format=json&length=767&ordering=name&start=0"

# 发送GET请求
response = requests.get(url)

# 检查请求是否成功
if response.status_code == 200:
    # 将响应的JSON内容保存到文件
    with open("stations_data.json", "w", encoding="utf-8") as json_file:
        json_file.write(response.text)
    print("文件已成功下载并保存为stations_data.json")
else:
    print(f"请求失败，状态码: {response.status_code}")


文件已成功下载并保存为stations_data.json


输入测站名称，自动获取经纬高信息

In [25]:
import json

#========================变量修改==============================
station_name = 'KAT1'
#==============================================================

#========================函数定义==============================
# 定义函数，根据name查找station的所有信息
def find_station_fullname_by_name(name):
    for station in stations_data:
        if station['name'][:4] == name:  # 确保station是字典，并匹配name
            return station['name']
    return None

# 定义函数，根据name查找llh信息
def find_llh_by_name(name):
    for station in stations_data:
        if station['name'] == name:  # 确保station是字典
            return station['llh']
    return None
#==============================================================

# 假设你已经正确读取了JSON文件
with open('/content/stations_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

stations_data = data.get('data', [])  # 根据实际键名替换 'stations'
station_full_name = find_station_fullname_by_name(station_name)  # 查找测站的完整名称
llh_info = find_llh_by_name(station_full_name)  # 查找对应完整名称的经纬高信息

if station_full_name:
    print(f"{station_name}的完整名称是: {station_full_name}")
else:
    print(f"未找到名称为 {station_name} 的站点。")

if llh_info:
    print(f"{station_name}的llh信息是: {llh_info}")
else:
    print(f"未找到名称为 {station_name} 的站点。")

KAT1的完整名称是: KAT100AUS
KAT1的llh信息是: [-14.375994444444443, 132.15327777777782, 184.3]


In [ ]:
import datetime
#####################################下载数据时的信息########################################
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2021, 12, 31)
station_name = 'KAT1'
obs_suffix = 'AUS_R'  # 新增的变量，用于替换 obs_filename 中的后缀

#######################################

#############################################处理数据时的信息#############################################
# AIRI 31.824, 130.600, 314.64
# BIK0 42.854, 74.533, 749.2
# BAIE 49.187, -68.263, 27.5
# CAS1 -66.283, 110.520, 22.6
# WUHN 30.532, 114.357, 28.2
# STFU 37.427, -122.173, 20.41
# HAL1 20.708, -156.256, 3049
# HKSL 22.372, 113.928, 95.3
# NTUS 1.346, 103.680, 79
# ANMG 2.785, 101.507, 15.002
# HLFX 44.684, -63.611, 3.1
# KAT1 -14.376, 132.153, 184.3
SS_variables = ['S2W']  # ['S1C', 'S1W', 'S2W']
station_llh = [-14.376, 132.153, 184.3]  # BAIE站点的经纬高
el_mask = 20  # 高度截止角
base_data_path = '/content/data' # 数据存储路径
start_year = start_date.year
end_year = end_date.year

####################################生成数据集时的信息####################################
# 定义起始年份和结束年份
signal = SS_variables[0]
# 定义卫星号的范围
satellites = [f'G{str(i).zfill(2)}' for i in range(1, 33)]  # 生成G01到G32的卫星号

#################################SFPD定义的信息====================================
# 可修改变量命名区
#years = ['2020','2021','2022']
years = ['2022']
#stations = ['AIRA', 'BAIE', 'BIK0', 'CAS1']
stations = ['HLFX']
svn_range = range(1, 33)  # G01到G32的范围

该部分下载耗时大概 5min/1year

In [ ]:

import pandas as pd
from ftplib import FTP_TLS, error_perm, error_temp
import gzip
import shutil
import os
import time

# 安装必要的库（如果还没安装）
!pip install pandas

# 函数：将日期转换为 GPS 周和周内的天数
def date_to_gps_week_day(date):
    gps_start = datetime.datetime(1980, 1, 6)
    delta = date - gps_start
    gps_week = delta.days // 7
    gps_day = delta.days % 7
    return gps_week, gps_day

# 函数：将日期转换为年份和DOY
def date_to_year_doy(date):
    year = date.year
    doy = (date - datetime.datetime(year, 1, 1)).days + 1
    return year, doy

# 函数：连接FTP服务器
def connect_to_ftp():
    print("正在初始化与FTP服务器的连接...")
    try:
        ftp = FTP_TLS('gdc.cddis.eosdis.nasa.gov')
        print("成功建立与FTP服务器的连接。")

        print("正在尝试登录...")
        ftp.login()
        print("登录成功。")

        print("正在切换到安全数据连接...")
        ftp.prot_p()  # 切换到加密的数据连接
        print("已建立安全数据连接。")

        return ftp
    except Exception as e:
        print(f"连接FTP服务器时出错: {e}")
        raise

# 函数：从FTP服务器下载文件并解压
def download_and_extract_files(start_date, end_date, save_path_obs, save_path_sp3, station_name, obs_suffix, download_obs=False, download_sp3=True):
    if not os.path.exists(save_path_obs):
        os.makedirs(save_path_obs)
        print(f"创建目录: {save_path_obs}")
    if not os.path.exists(save_path_sp3):
        os.makedirs(save_path_sp3)
        print(f"创建目录: {save_path_sp3}")

    # 连接FTP服务器
    print("正在连接FTP服务器...")
    ftp = connect_to_ftp()
    print("FTP连接成功建立。")

    day_count = 0
    start_time = time.time()

    current_year = start_date.year

    # 遍历日期范围
    for single_date in pd.date_range(start=start_date, end=end_date):
        print(f"\n正在处理日期: {single_date.strftime('%Y-%m-%d')}")

        if single_date.year != current_year:
            current_year = single_date.year
            save_path_obs = f"/content/data/obs/{station_name}/{current_year}"
            save_path_sp3 = f'/content/data/sp3/{current_year}'
            if not os.path.exists(save_path_obs):
                os.makedirs(save_path_obs)
                print(f"为obs数据创建新目录: {save_path_obs}")
            if not os.path.exists(save_path_sp3):
                os.makedirs(save_path_sp3)
                print(f"为sp3数据创建新目录: {save_path_sp3}")

        gps_week, gps_day = date_to_gps_week_day(single_date)
        year, doy = date_to_year_doy(single_date)
        year_short = single_date.strftime('%y')
        doy_padded = f'{doy:03d}'

        # 下载obs文件
        if download_obs:
            obs_filename = f'{station_name}00{obs_suffix}_{year}{doy_padded}0000_01D_30S_MO.crx.gz'
            obs_directory = f'/pub/gps/data/daily/{year}/{doy_padded}/{year_short}d/'
            local_obs_gz_filename = os.path.join(save_path_obs, obs_filename)
            local_obs_filename = os.path.join(save_path_obs, obs_filename[:-3])

            # 检查文件是否已经存在
            if os.path.exists(local_obs_filename.replace(".crx", ".crx")):
                print(f'{local_obs_filename.replace(".crx", ".crx")} 已存在，跳过下载。')
            else:
                max_retries = 3
                retry_delay = 5
                for attempt in range(max_retries):
                    try:
                        print(f"正在下载 {obs_filename}...")
                        with open(local_obs_gz_filename, 'wb') as file:
                            ftp.retrbinary(f'RETR {obs_directory}{obs_filename}', file.write)
                        print(f'成功下载 {local_obs_gz_filename}')

                        print(f"正在解压 {local_obs_gz_filename}...")
                        with gzip.open(local_obs_gz_filename, 'rb') as f_in:
                            with open(local_obs_filename, 'wb') as f_out:
                                shutil.copyfileobj(f_in, f_out)
                        print(f'成功解压到 {local_obs_filename}')

                        # 删除压缩文件
                        os.remove(local_obs_gz_filename)
                        print(f"删除压缩文件 {local_obs_gz_filename}")
                        break
                    except error_perm as e:
                        if str(e).startswith('550'):  # File not found
                            print(f"服务器上未找到文件 {obs_filename}，跳过。")
                            break
                        else:
                            raise
                    except error_temp as e:
                        if attempt < max_retries - 1:
                            print(f"临时错误: {e}。{retry_delay} 秒后重试...")
                            time.sleep(retry_delay)
                        else:
                            print(f"尝试 {max_retries} 次后仍无法下载 {obs_filename}。")
                            break
                    except Exception as e:
                        if 'EOF occurred in violation of protocol (_ssl.c:2427)' in str(e):
                            print('发生EOF错误，正在重新连接FTP服务器...')
                            ftp = connect_to_ftp()
                        else:
                            print(f'处理 {local_obs_gz_filename} 时出错: {e}')
                            break

        # 下载sp3文件
        if download_sp3:
            sp3_filename = f'COD0MGXFIN_{year}{doy_padded}0000_01D_05M_ORB.SP3.gz'
            sp3_directory = f'/pub/gps/products/{gps_week}/'
            local_sp3_gz_filename = os.path.join(save_path_sp3, sp3_filename)
            local_sp3_filename = os.path.join(save_path_sp3, f'COD0MGXFIN_{year}{doy_padded}0000_01D_05M_ORB.SP3')

            # 检查文件是否已经存在
            if os.path.exists(local_sp3_filename):
                print(f'{local_sp3_filename} 已存在，跳过下载。')
            else:
                max_retries = 3
                retry_delay = 5
                for attempt in range(max_retries):
                    try:
                        print(f"正在下载 {sp3_filename}...")
                        with open(local_sp3_gz_filename, 'wb') as file:
                            ftp.retrbinary(f'RETR {sp3_directory}{sp3_filename}', file.write)
                        print(f'成功下载 {local_sp3_gz_filename}')

                        print(f"正在解压 {local_sp3_gz_filename}...")
                        with gzip.open(local_sp3_gz_filename, 'rb') as f_in:
                            with open(local_sp3_filename, 'wb') as f_out:
                                shutil.copyfileobj(f_in, f_out)
                        print(f'成功解压到 {local_sp3_filename}')

                        # 删除压缩文件
                        os.remove(local_sp3_gz_filename)
                        print(f"删除压缩文件 {local_sp3_gz_filename}")
                        break
                    except error_perm as e:
                        if str(e).startswith('550'):  # File not found
                            print(f"服务器上未找到文件 {sp3_filename}，跳过。")
                            break
                        else:
                            raise
                    except error_temp as e:
                        if attempt < max_retries - 1:
                            print(f"临时错误: {e}。{retry_delay} 秒后重试...")
                            time.sleep(retry_delay)
                        else:
                            print(f"尝试 {max_retries} 次后仍无法下载 {sp3_filename}。")
                            break
                    except Exception as e:
                        if 'EOF occurred in violation of protocol (_ssl.c:2427)' in str(e):
                            print('发生EOF错误，正在重新连接FTP服务器...')
                            ftp = connect_to_ftp()
                        else:
                            print(f'处理 {local_sp3_gz_filename} 时出错: {e}')
                            break

        day_count += 1

        # 每下载10天的文件输出一次计时信息
        if day_count % 10 == 0:
            elapsed_time = time.time() - start_time
            print(f'已下载 {day_count} 天的文件，耗时: {elapsed_time:.2f} 秒')

    # 断开FTP连接
    print("正在关闭FTP连接...")
    ftp.quit()
    print("FTP连接已成功关闭。")

In [ ]:
# 示例使用
if __name__ == "__main__":
    save_path_obs = f"/content/data/obs/{station_name}/{start_date.year}"
    save_path_sp3 = f'/content/data/sp3/{start_date.year}'

    print("开始下载和提取文件...")
    download_and_extract_files(start_date, end_date, save_path_obs, save_path_sp3, station_name, obs_suffix, download_obs=True, download_sp3=True)
    print("下载和提取过程完成。")

流式输出内容被截断，只能显示最后 5000 行内容。
成功下载 /content/data/obs/KAT1/2020/KAT100AUS_R_20203190000_01D_30S_MO.crx.gz
正在解压 /content/data/obs/KAT1/2020/KAT100AUS_R_20203190000_01D_30S_MO.crx.gz...
成功解压到 /content/data/obs/KAT1/2020/KAT100AUS_R_20203190000_01D_30S_MO.crx
删除压缩文件 /content/data/obs/KAT1/2020/KAT100AUS_R_20203190000_01D_30S_MO.crx.gz
正在下载 COD0MGXFIN_20203190000_01D_05M_ORB.SP3.gz...
成功下载 /content/data/sp3/2020/COD0MGXFIN_20203190000_01D_05M_ORB.SP3.gz
正在解压 /content/data/sp3/2020/COD0MGXFIN_20203190000_01D_05M_ORB.SP3.gz...
成功解压到 /content/data/sp3/2020/COD0MGXFIN_20203190000_01D_05M_ORB.SP3
删除压缩文件 /content/data/sp3/2020/COD0MGXFIN_20203190000_01D_05M_ORB.SP3.gz

正在处理日期: 2020-11-15
正在下载 KAT100AUS_R_20203200000_01D_30S_MO.crx.gz...
成功下载 /content/data/obs/KAT1/2020/KAT100AUS_R_20203200000_01D_30S_MO.crx.gz
正在解压 /content/data/obs/KAT1/2020/KAT100AUS_R_20203200000_01D_30S_MO.crx.gz...
成功解压到 /content/data/obs/KAT1/2020/KAT100AUS_R_20203200000_01D_30S_MO.crx
删除压缩文件 /content/data/obs/KAT1/2020/KAT100

检查文件夹/content/data/obs/下的所有子文件（包括子文件夹中的文件）是否有.gz结尾的，如果有，删除这些文件

In [ ]:
import os

# 指定文件夹路径
folder_path = '/content/data/obs/'

# 存储被删除的文件
deleted_files = []

# 遍历文件夹及其子文件夹
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.gz'):
            # 构造完整文件路径
            file_path = os.path.join(root, file)
            # 删除文件
            os.remove(file_path)
            # 记录被删除的文件
            deleted_files.append(file_path)

# 打印删除的文件数量和名称
print(f'删除的文件数量: {len(deleted_files)}')
print('删除的文件:')
for deleted_file in deleted_files:
    print(deleted_file)


删除的文件数量: 1
删除的文件:
/content/data/obs/KAT1/2020/KAT100AUS_R_20202900000_01D_30S_MO.crx.gz


/content/data/obs/下的所有子文件（包括子文件夹中的文件）.crx，全部使用crx2rnx.exe进行转换

下载crx2rnx工具，并解压

In [ ]:
import os

# 定义目标文件夹路径
target_dir = '/content/tool'

# 创建目标文件夹（如果不存在）
os.makedirs(target_dir, exist_ok=True)

# 定义下载链接和目标文件路径
download_url = 'https://terras.gsi.go.jp/ja/crx2rnx/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz'
tar_file_path = os.path.join(target_dir, 'RNXCMP_4.1.0_Linux_x86_32bit.tar.gz')

# 使用 wget 下载文件
!wget -P {target_dir} {download_url}

# 解压缩文件
!tar -xzf {tar_file_path} -C {target_dir}

# 删除 .tar.gz 文件
os.remove(tar_file_path)

print(f"文件已下载并解压到 {target_dir} 并删除了 {tar_file_path}")

--2024-10-08 13:57:59--  https://terras.gsi.go.jp/ja/crx2rnx/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz
Resolving terras.gsi.go.jp (terras.gsi.go.jp)... 163.42.5.1
Connecting to terras.gsi.go.jp (terras.gsi.go.jp)|163.42.5.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 497324 (486K) [application/x-gzip]
Saving to: ‘/content/tool/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz’

RNXCMP_4.1.0_Linux_ 100%[===================>] 485.67K   748KB/s    in 0.6s    

2024-10-08 13:58:00 (748 KB/s) - ‘/content/tool/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz’ saved [497324/497324]

文件已下载并解压到 /content/tool 并删除了 /content/tool/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz


对所有data/obs下的所有子文件夹下的所有子文件，调用"/content/tool/RNXCMP_4.1.0_Linux_x86_32bit/bin/CRX2RNX {filename}"

此步耗时8min/2year。

In [ ]:
import os
import subprocess

# 定义工具路径和数据目录路径
tool_path = '/content/tool/RNXCMP_4.1.0_Linux_x86_32bit/bin/CRX2RNX'
data_dir = '/content/data/obs'

# 遍历目录及其子目录
for root, dirs, files in os.walk(data_dir):
    for file in files:
        # 检查文件是否以 .crx 结尾
        if file.endswith('.crx'):
            # 构建文件路径
            file_path = os.path.join(root, file)
            rnx_file_path = os.path.join(root, file[:-4] + '.rnx')

            # 检查是否已经存在对应的 .rnx 文件
            if os.path.exists(rnx_file_path):
                print(f"Skipped: {file_path} (already converted to {rnx_file_path})")
                continue  # 跳过转换

            try:
                # 调用 CRX2RNX 工具
                command = f'{tool_path} {file_path}'
                subprocess.run(command, shell=True, check=True)
                print(f"Processed: {file_path}")
            except subprocess.CalledProcessError as e:
                print(f"Error processing {file_path}: {e}")
                continue  # 跳过错误并继续处理下一个文件

# 删除所有 .crx 文件
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.crx'):
            file_path = os.path.join(root, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")

Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20212870000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20211150000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20213540000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20210240000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20211690000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20210980000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20212150000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20210710000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20211050000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20213050000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20210650000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20211040000_01D_30S_MO.crx
Processed: /content/data/obs/KAT1/2021/KAT100AUS_R_20210890000_0

# 2 处理得到载噪比数据

安装python需要的包

In [ ]:
!pip install pyunpack patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 2.4 MB/s eta 0:00:00


下载项目，实现调用dataprocessing的代码

In [ ]:
!git clone https://github.com/BlackiePiggy/SFPD.git

fatal: destination path 'SFPD' already exists and is not an empty directory.


执行载噪比处理程序

读取obs文件耗时大概为200天/11minutes

总耗时约1hour/1year

In [ ]:
import sys
import os
# 将 /content/dataproc 文件夹添加到 Python 路径
sys.path.append('/content/SFPD/dataprocessing')
sys.path.append('/content/SFPD/dataprocessing/lib')

import read_CN_from_obs as readCN
import read_el_from_sp3 as readEL
import combine_cn_el as combineCNEL
import elFilter as elft

################################################################################################
def generate_satellite_range(prefix, start, end):
    return [f'{prefix}{str(i).zfill(2)}' for i in range(start, end + 1)]

satellite_range = generate_satellite_range('G', 1, 32)  # 生成要处理的卫星范围，可自选

for year in range(start_year, end_year+1):  # 遍历2022到2023年
    test_start_date = f'{year}001'
    test_end_date = f'{year}365'

    #########################################文件路径################################################
    test_input_obs_folder = f'{base_data_path}/obs/{station_name}/{year}'  # 替换为你的输入数据目录路径
    test_input_sp3_dir = f'{base_data_path}/sp3/{year}'
    test_output_cn_folder = f"{base_data_path}/1_time_CN/{year}"  # 替换为你的输出数据目录路径
    test_output_el_folder = f'{base_data_path}/2_time_EL/{year}'
    test_output_cn_el_folder = f'{base_data_path}/3_time_CN_EL/{year}'
    test_time_cn_el_filtered_folder = f'{base_data_path}/4_filtered_time_CN/{year}'
    ######################################################################

    # 第一步：读取所有obs文件，提取CN值，存储到timeCN文件夹中
    readCN.read_CN_value_from_obs_AAO(station_name, SS_variables, 'all', test_input_obs_folder, test_output_cn_folder, test_start_date, test_end_date)  # 正常日期

    readEL.read_el_from_sp3(station_name, test_input_sp3_dir, test_output_el_folder, satellite_range, station_llh, test_start_date, test_end_date)

    combineCNEL.merge_cn_el_files(station_name, SS_variables, satellite_range, test_output_cn_folder, test_output_el_folder, test_output_cn_el_folder, test_start_date, test_end_date)  # 正常日期

    elft.filter_cn_el_files(station_name, SS_variables, satellite_range, test_output_cn_el_folder, test_time_cn_el_filtered_folder, test_start_date, test_end_date, el_mask)  # 正常日期

    print(f"Test Dataset for year {year} Generated Successfully!")
    ######################################################################


Streaming output truncated to the last 5000 lines.
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021209_CN_G27_S2W_el_20_filtered.csv
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021209_CN_G28_S2W_el_20_filtered.csv
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021209_CN_G29_S2W_el_20_filtered.csv
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021209_CN_G30_S2W_el_20_filtered.csv
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021209_CN_G31_S2W_el_20_filtered.csv
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021209_CN_G32_S2W_el_20_filtered.csv
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021210_CN_G01_S2W_el_20_filtered.csv
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021210_CN_G02_S2W_el_20_filtered.csv
Saved filtered file: /content/data/4_filtered_time_CN/2021/KAT1_2021210_CN_G03_S2W_el_20_filtered.csv
Saved filtered file: /content/d

生成数据集，最终生成数据集存放于data/dataset文件夹中

运行时间：1min/1year

In [ ]:
import os
import pandas as pd


# 遍历起始年份和结束年份之间的每一年
for year in range(start_year, end_year + 1):
    year = str(year)  # 转换年份为字符串
    date_start = f'{year}001'
    date_end = f'{year}365'

    # 定义输入输出路径
    test_data_path = rf'/content/data/4_filtered_time_CN/{year}'
    output_base_path = rf'/content/drive/MyDrive/Colab Notebooks/SFPD/dataset/{year}/{station_name}'

    for satellite in satellites:
        # 动态设置输出路径
        output_path = os.path.join(output_base_path, f'{station_name}_{year}_{signal}_{satellite}.csv')

        # 创建一个空的列表来存储数据框
        data_frames = []

        # 遍历所有文件
        for path in [test_data_path]:
            for filename in os.listdir(path):
                if filename.startswith(station_name) and filename.endswith('_filtered.csv'):
                    parts = filename.split('_')
                    date_str = parts[1]
                    satellite_str = parts[3]
                    signal_str = parts[4]

                    # 确保文件的日期在范围内，并匹配卫星号和信号
                    if date_start <= date_str <= date_end and satellite_str == satellite and signal_str == signal:
                        file_path = os.path.join(path, filename)
                        df = pd.read_csv(file_path)
                        data_frames.append(df)

        # 如果有符合条件的数据框则进行处理
        if data_frames:
            # 合并所有数据框
            merged_df = pd.concat(data_frames, ignore_index=True)

            # 删除第三列
            merged_df.drop(merged_df.columns[2], axis=1, inplace=True)

            # 确认Timestamp列存在并进行格式转换
            if 'Timestamp' in merged_df.columns:
                merged_df['Timestamp'] = pd.to_datetime(merged_df['Timestamp'])

            # 按日期排序
            merged_df.sort_values(by='Timestamp', inplace=True)

            # 输出合并后的数据框到指定文件
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            merged_df.to_csv(output_path, index=False)

            print(f"合并后的文件已保存到: {output_path}")
        else:
            print(f"没有找到符合条件的文件：年份 {year} 卫星 {satellite}")


# 3 SFPD检测算法

安装必要的python包

In [ ]:
!pip install fastdtw

执行SFPD算法

运行速度：

In [ ]:
# main.py
import sys
import os
# 将 /content/dataproc 文件夹添加到 Python 路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks/SFPD/src')

from sfpd import calculate_sfpd


timestamp_format = '%Y-%m-%d %H:%M:%S'
residual_threshold = 5
save_plots = True  # 控制是否保存每日图像的布尔变量
figsize = (15, 5)  # 图形的大小 (宽, 高)
plot_dpi = 100  # 图形的分辨率
input_data_path = '/content/drive/MyDrive/Colab Notebooks/SFPD/dataset'  # 输入CSV文件所在目录

# 自定义输出路径
output_image_dir_base = '/content/drive/MyDrive/Colab Notebooks/SFPD/DTW_plot_result'  # 图像输出基础路径
output_DTW_dir_base = '/content/drive/MyDrive/Colab Notebooks/SFPD/DTW_result'  # DTW输出路径

# 调用SFPD计算函数
calculate_sfpd(years, stations, SS_variables, svn_range, timestamp_format, residual_threshold, save_plots, figsize, plot_dpi,
               input_data_path, output_image_dir_base, output_DTW_dir_base)
